In [ ]:
import sys
import math
from heapq import *
from collections import defaultdict
import random
import networkx as nx
import matplotlib.pyplot as plt
from typing import List
sys.setrecursionlimit(10**7)

---
### グラフの作成

In [ ]:
n = 10

# 隣接リストの作成
to = [[] for _ in range(n*n)]
for i in range(n*n):
    if (i+1) % n != 0:
        to[i].append(i+1)
    if i % n != 0:
        to[i].append(i-1)
    if i+n < n*n:
        to[i].append(i+n)
    if i-n >= 0:
        to[i].append(i-n)

In [ ]:
# 通過不可能領域の生成
def generate_obstruction() -> List:
    """
    通過不可能領域は矩形とする。
    10×10
    """
    area_1 = [12, 13, 22, 23, 32, 33]
    area_2 = [65, 66, 67, 75, 76, 77]
    obs_area_list = area_1 + area_2

    obs_area_dict = defaultdict(int)
    for e in obs_area_list:
        obs_area_dict[e] = 1

    return obs_area_list, obs_area_dict


In [ ]:
# グラフの可視化
def vis_gridpath(n, path_nodes, obs_nodes=None):
    
    # n×n の格子グラフを作成
    G = nx.grid_2d_graph(n, n)
    mapping = {(i, j): i*n + j for i, j in G.nodes()}
    G = nx.relabel_nodes(G, mapping)
    pos = {i * n + j: (j, -i) for i in range(n) for j in range(n)}
    path_edges = list(zip(path_nodes[:-1], path_nodes[1:]))

    # 描画
    plt.figure(figsize=(6,6))

    # 通常ノード・エッジ
    nx.draw_networkx_nodes(G, pos, node_color='lightgray', node_size=500)
    nx.draw_networkx_edges(G, pos, edge_color='gray')
    nx.draw_networkx_labels(G, pos)

    # 障害物エリアの描画
    if obs_nodes:
        nx.draw_networkx_nodes(G, pos, nodelist=obs_nodes, node_color='black', node_size=500)
        labels = {n: n for n in obs_nodes}
        nx.draw_networkx_labels(
            G, pos,
            labels=labels,
            font_color='white',
        )

    # 経路部分を強調（色を変える）
    nx.draw_networkx_nodes(G, pos, nodelist=path_nodes, node_color='orange')
    nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='red', width=2)

    plt.axis('off')
    plt.show()

In [ ]:
obs_nodes_list, obs_nodes_dict = generate_obstruction()

n = 10
path_nodes = [0, 1, 2, 3, 4, 5, 6, 7]
vis_gridpath(n, path_nodes, obs_nodes_list)

In [ ]:
n = 10
path_nodes = [0, 1, 2, 12, 13, 14, 24]
vis_gridpath(n, path_nodes)

---
## DFSその2
頂点vからスタートし、事前に決めたゴール地点に長さLで進める経路をすべて列挙する。


In [ ]:
class DFS:
    def __init__(self, to, obs_nodes, grid_size, start, goal, length):
        self.to = to
        self.obs_nodes = obs_nodes
        self.grid_size = grid_size
        self.start = start
        self.goal = goal
        self.length = length
        self.used = [False for _ in range(grid_size*grid_size)]
        self.path = []
        self.anss = []

    def _dfs_2(self, v):
        self.path.append(v)
        self.used[v] = True    
        if len(self.path) >= self.length+1:
            if self.path[-1] == self.goal:
                self.anss.append(self.path.copy())
            self.path.remove(v)
            self.used[v] = False
            return 
        for u in self.to[v]:
            # 障害ノードは避ける
            if u in self.obs_nodes:
                continue
            if not self.used[u]:
                self._dfs_2(u)
        self.path.remove(v)
        self.used[v] = False

    def exec(self):
        self._dfs_2(self.start)



In [ ]:
dfs = DFS(to, obs_nodes_list, n, 0, 86, 24)
dfs.exec()
print(len(dfs.anss))

In [ ]:
vis_gridpath(10, dfs.anss[4], obs_nodes=obs_nodes_list)

---
### 経路保持

In [ ]:
length = 14

path_info = {
    'No.1':{
        'start': 0,
        'goal': 46,
        'length': length,
    },
    'No.2':{
        'start': 21,
        'goal': 38,
        'length': length,
    },
    'No.3':{
        'start': 26,
        'goal': 73,
        'length': length,
    },
}

In [ ]:
dfs_path1 = DFS(to, obs_nodes_list, n, path_info['No.1']['start'], path_info['No.1']['goal'], path_info['No.1']['length'])
dfs_path2 = DFS(to, obs_nodes_list, n, path_info['No.2']['start'], path_info['No.2']['goal'], path_info['No.2']['length'])
dfs_path3 = DFS(to, obs_nodes_list, n, path_info['No.3']['start'], path_info['No.3']['goal'], path_info['No.3']['length'])

dfs_path1.exec()
dfs_path2.exec()
dfs_path3.exec()

In [ ]:
print(len(dfs_path1.anss))
print(len(dfs_path2.anss))
print(len(dfs_path3.anss))

---
### 方針案1実装(その1)
- pymoo不使用ver

---
### 方針案1実装(その2)
- pymooを使って最適化
- 

In [ ]:
class Policy_1:
    def __init__(self, paths1, paths2, paths3):
        self.paths1 = paths1
        self.paths2 = paths2
        self.paths3 = paths3
        
    def single_point_crossover(self, parent1, parent2):
        """
        一点交叉(single-point crossover)

        Args:
        Returns:
        """
        point = random.randint(1, N - 1)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2
    

    def uniform_crossover(self, parent1, parent2):
        """
        一様交叉(uniform crossover)

        Args:
        Returns:
        """
        child = []
        for i in range(len(parent1)):
            if random.random() < 0.5:
                child.append(parent1[i])
            else:
                child.append(parent2[i])
        return child


    def mutate(self, chromosome, mutation_rate, route_options_per_wire):
        """
        突然変異(mutation)

        Args:
        Returns:
        """
        new_chromosome = chromosome[:]
        for i in range(len(chromosome)):
            if random.random() < mutation_rate:
                k = len(route_options_per_wire[i])
                new_idx = random.randint(0, k - 1)
                while new_idx == new_chromosome[i]:  # 同じならもう一度
                    new_idx = random.randint(0, k - 1)
                new_chromosome[i] = new_idx
        return new_chromosome
    

    def fitness(self, chromosome):
        """
        目的関数
        """

        paths = [route_options_per_wire[i][chromosome[i]] for i in range(N)]
        total_conflict = 0
        occupied = defaultdict(int)
        for path in paths:
            for node in path:
                occupied[node] += 1
        for node, count in occupied.items():
            if count > 1:
                total_conflict += count - 1  # or (count choose 2)
        return -total_conflict  # 小さいほど良い



---
## GAの練習

In [ ]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
from pymoo.termination import get_termination

# 1. 問題の定義
class MyProblem(ElementwiseProblem):
    def __init__(self):
        super().__init__(n_var=2, n_obj=1, n_constr=0, xl=np.array([-5, -5]), xu=np.array([5, 5]))

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = x[0]**2 + x[1]**2
        out["F"] = f1

# 2. 問題のインスタンスを作成
problem = MyProblem()

# 3. アルゴリズムの選択と設定
algorithm = GA(pop_size=20, eliminate_duplicates=True) # 各世代で20個体(解)を保持する。

# 4. 最適化の実行
termination = get_termination("n_gen", 100) # 100世代まわす
res = minimize(problem, algorithm, termination, seed=1, verbose=True)

# 5. 結果の表示
print("最適化された設計変数: %s" % res.X)
print("最小化された目的関数値: %s" % res.F)

### TSPをGAで解く

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.ga import GA
# from pymoo.operators.sampling.perm_random import PermutationRandomSampling
from pymoo.operators.sampling.rnd import PermutationRandomSampling
from pymoo.operators.crossover.ox import OrderCrossover
from pymoo.operators.mutation.inversion import InversionMutation
from pymoo.optimize import minimize

# 都市数
N = 20

# 各都市の座標（ランダムに生成）
np.random.seed(42)
cities = np.random.rand(N, 2)

# 巡回セールスマン問題を定義
class TSP(ElementwiseProblem):
    def __init__(self, cities):
        super().__init__(
            n_var = len(cities),
            n_obj = 1,
            n_constr = 0,
            xl = 0,
            xu = len(cities)-1,
            type_var = np.int64,
            elementwise_evaluation = True
        )
        self.cities = cities

    def _evaluate(self, x, out, *args, **kwargs):
        '''
        Arg:
            x : 決定変数。今の場合、xは「都市の訪問順を表す長さNの順列」を表す。
        '''
        # 巡回経路の距離を計算
        dist = 0.0
        for i in range(len(x)):
            a = self.cities[x[i]]
            b = self.cities[x[(i+1) % len(x)]]
            dist += np.linalg.norm(a-b)
        out['F'] = dist

problem = TSP(cities)

# GAアルゴリズムの構成（順列用）
algorithm = GA(
    pop_size = 100, # 集団サイズ（個体数）
    sampling = PermutationRandomSampling(), # 都市の訪問順列の初期サンプリング
    crossover = OrderCrossover(), # TSPに適した交叉（順序保持型交叉）
    mutation = InversionMutation(), # 順列に対する突然変異オペレータ
    eliminate_duplicates=True, # 重複排除。個体ごとに評価を行う設定（順列にはこれが必要） 
)

# 最適化の実行
res = minimize(
    problem,
    algorithm,
    termination=('n_gen', 200),  # 200世代
    seed=1,
    verbose=True
)

# 最良解の表示
print("Best tour:", res.X)
print("Best distance:", res.F[0])


# 結果の可視化
def plot_tour(order, cities):
    path = cities[order]
    path = np.vstack([path, path[0]])  # 巡回するために始点を最後に加える
    plt.plot(path[:, 0], path[:, 1], marker='o')
    plt.title("Best TSP Tour")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.grid(True)
    plt.show()


plot_tour(res.X, cities)
